# A Broken Neural Network 

I could have tried to find a solution to this challenge using a Random Forest / XGBoost approach, but I decided to use a neural network instead because I'm a complete deep learning newbie. Since I never used that technique for regression problems, I thought it could be fun to give it a try, but after a few hours of hyper parameters tweaking and loss functions testing, my network still doesn't work (at all). Is it the loss function choice that is wrong? Is it the shape of the graph? Is it a standardization problem? Is it just the wrong approach for that kind of problem? I have no idea.

Do you folks know what went wrong? Any feedback is much appreciated!

## Loading & Cleaning

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from geopy.distance import vincenty

In [4]:
train = pd.read_csv('../input/train.csv')

In [5]:
test = pd.read_csv('../input/test.csv')

### Outliers

In [6]:
# new york city area
min_lon = -74.844,
min_lat = 40.026
max_lon = -72.221
max_lat = 41.372

In [7]:
initial_len = train.shape[0]

In [8]:
# ruling points outside area as outliers
train = train[train['pickup_longitude'].between(min_lon, max_lon)]
train = train[train['pickup_latitude'].between(min_lat, max_lat)]
train = train[train['dropoff_longitude'].between(min_lon, max_lon)]
train = train[train['dropoff_latitude'].between(min_lat, max_lat)]

In [9]:
cleaned_len = train.shape[0]

In [10]:
# outliers removed
initial_len - cleaned_len

44

### Compute distance

In [11]:
def get_distance(row):
    p1 = (row['pickup_latitude'], row['pickup_longitude'])
    p2 = (row['dropoff_latitude'], row['dropoff_longitude'])
    return vincenty(p1, p2).meters

In [12]:
train['distance'] = train.apply(get_distance, axis=1)

In [13]:
lon = train.loc[0]['pickup_longitude']
lat = train.loc[0]['pickup_latitude']

### NaN Correction

In [14]:
# check for na's
train.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
distance              0
dtype: int64

### Datetime

In [15]:
# to datetime
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime'] = pd.to_datetime(train['dropoff_datetime'])

In [16]:
# get day and hour information
train['pickup_hour'] = train['pickup_datetime'].apply(lambda x: x.hour)
train['pickup_day'] = train['pickup_datetime'].apply(lambda x: x.weekday())

In [17]:
# convert timestamps to float
train['pickup_datetime'] = train['pickup_datetime'].apply(lambda x: x.timestamp())
train['dropoff_datetime'] = train['dropoff_datetime'].apply(lambda x: x.timestamp())

In [18]:
# get dummy variables for categorial data
train['store_and_fwd_flag'] = pd.get_dummies(train['store_and_fwd_flag'])

### Irrelevant features

In [19]:
train = train.drop('id', axis=1)

## Exploration

### Distance / Trip Duration

In [20]:
from scipy.stats import pearsonr

In [21]:
pearsonr(train['distance'], train['trip_duration'])

(0.10287229740259092, 0.0)

## RNN

In [22]:
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, LSTM, Merge
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [23]:
X = train.drop('trip_duration', axis=1)
y = train['trip_duration']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [25]:
y_norm = normalize(y.reshape(1, -1))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [26]:
X_train.shape

(1093950, 12)

In [27]:
y_norm.T.shape

(1458600, 1)

In [28]:
def baseline():
    model = Sequential()
    model.add(Dense(48, input_dim=12, activation='relu', kernel_initializer='normal'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='linear', kernel_initializer='normal'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

In [29]:
estimators = []
regressor = KerasRegressor(build_fn=baseline, epochs=5, batch_size=50, verbose=1)
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', regressor))
pipeline = Pipeline(estimators)

In [30]:
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, y_norm.T, cv=kfold)

Epoch 1/5
1166880/1166880 [==============================] - 61s - loss: 8.6363e-07 - acc: 0.0000e+00    
Epoch 2/5
1166880/1166880 [==============================] - 61s - loss: 8.0243e-07 - acc: 0.0000e+00    
Epoch 3/5
1166880/1166880 [==============================] - 47s - loss: 8.0432e-07 - acc: 0.0000e+00    
Epoch 4/5
1166880/1166880 [==============================] - 51s - loss: 8.0250e-07 - acc: 0.0000e+00    
Epoch 5/5
 675500/1166880 [================>.............] - ETA: 20s - loss: 1.8000e-06 - acc: 0.0000e+00

In [31]:
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -0.00 (0.00) MSE
